In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import html
from tqdm import tqdm
tqdm.pandas()

In [2]:
train_df  = pd.read_csv('/content/drive/MyDrive/Collab/Введение в обработку естественного языка/Homework01/train_tweets.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Collab/Введение в обработку естественного языка/Homework01/test_tweets.csv')


In [3]:
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


Для работы объединим train_df и test_df

In [4]:
data_df = train_df.append(test_df, ignore_index=True)

Заменим html-сущности (к примеру: < > &). "<" заменим на “<” и "&" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'


In [5]:
import html
from tqdm import tqdm
tqdm.pandas()

In [6]:
def html_parser(line):
    line = html.unescape(line)
    return line

In [7]:
data_df['tidy_tweet'] = data_df.tweet.progress_apply(lambda x: html_parser(x))

100%|██████████| 49159/49159 [00:00<00:00, 592526.00it/s]


In [8]:
#data_df

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation
...,...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads...","happy, at work conference: right mindset leads..."


Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:

*   для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
*   для для замены @user на пробел, необходимо использовать re.sub() при применении функции необходимо использовать np.vectorize(function).



In [9]:
import re
import numpy as np

In [10]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [11]:
data_df['tidy_tweet'] = np.vectorize(remove_pattern)(data_df['tidy_tweet'], "@[\w]*")

Изменим регистр твитов на нижний с помощью .lower().

In [12]:
data_df['tidy_tweet'] = data_df['tidy_tweet'].str.lower()

Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).


In [13]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [14]:
def replace_dict(word, dictionary):
    return dictionary.get(word, word)
    
def preprocess_apostrophe(line, apostrophe_dict=apostrophe_dict):
    line = " ".join(replace_dict(word, apostrophe_dict) for word in line.split())
    return line

In [15]:
data_df['tidy_tweet'] = data_df.tidy_tweet.progress_apply(lambda x: preprocess_apostrophe(x))

100%|██████████| 49159/49159 [00:00<00:00, 113006.36it/s]


Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [16]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [17]:
def preprocess_short(line, short_word_dict=short_word_dict):
    line = " ".join(replace_dict(word, short_word_dict) for word in line.split())
    return line

In [18]:
data_df['tidy_tweet'] = data_df.tidy_tweet.progress_apply(lambda x: preprocess_short(x))

100%|██████████| 49159/49159 [00:00<00:00, 83478.90it/s]


Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [19]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [20]:
def preprocess_emotion(line, emoticon_dict=emoticon_dict):
    line = " ".join(replace_dict(word, emoticon_dict) for word in line.split())
    return line

In [21]:
data_df['tidy_tweet'] = data_df.tidy_tweet.progress_apply(lambda x: preprocess_emotion(x))

100%|██████████| 49159/49159 [00:00<00:00, 79844.37it/s]


Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

In [22]:
data_df['tidy_tweet'] = data_df['tidy_tweet'].str.replace("[^\w\s]", " ")

<ipython-input-22-0a02f7a6cf9d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['tidy_tweet'] = data_df['tidy_tweet'].str.replace("[^\w\s]", " ")


Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

In [23]:
data_df['tidy_tweet'] = data_df['tidy_tweet'].str.replace("[^a-zA-Z0-9]", " ")

<ipython-input-23-d97aca44eda7>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['tidy_tweet'] = data_df['tidy_tweet'].str.replace("[^a-zA-Z0-9]", " ")


Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.


In [24]:
data_df['tidy_tweet'] = data_df['tidy_tweet'].str.replace("[^a-zA-Z]", " ")

<ipython-input-24-5f8a7d60ba2e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['tidy_tweet'] = data_df['tidy_tweet'].str.replace("[^a-zA-Z]", " ")


Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [25]:
data_df['tidy_tweet'] = data_df['tidy_tweet'].progress_apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

100%|██████████| 49159/49159 [00:00<00:00, 143448.66it/s]


In [26]:
#Cоберем в единую функцию

def replace_dict(word, dictionary):
    return dictionary.get(word, word)

def preprocess(line, apostrophe_dict=apostrophe_dict, short_word_dict=short_word_dict, emoticon_dict=emoticon_dict):
#     Заменим html-сущности (к примеру: < > &). "<" заменим на “<” и "&" заменим на “&”)""". 
#     Сделаем это с помощью HTMLParser.unescape()
    line = html.unescape(line)
    
#     Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:
#     для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
#     для для замены @user на пробел, необходимо использовать re.sub() при применении функции 
#     необходимо использовать np.vectorize(function)
    line = re.sub(r'@[\w]*','', line)
    
    #Изменим регистр твитов на нижний с помощью .lower()
    line = line.lower()
    
#     Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. 
#     Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), 
#     если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), 
#     то заменить ключ на значение (полную версию слова).

    line = " ".join(replace_dict(word, apostrophe_dict) for word in line.split())
    
#     Заменим сокращения на их полные формы, используя short_word_dict. 
#     Для этого воспользуемся функцией, используемой в предыдущем пункте.

    line = " ".join(replace_dict(word, short_word_dict) for word in line.split())
    
#     Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. 
#     Для этого воспользуемся функцией, используемой в предыдущем пункте.

    line = " ".join(replace_dict(word, emoticon_dict) for word in line.split())
    
#     Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

    line = re.sub(r'[^\w\s]',' ', line)

#     Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

    line = re.sub(r'[^a-zA-Z0-9]', ' ', line)

#     Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

    line = re.sub(r'[^a-zA-Z]', ' ', line)
    
#     Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

    line = ' '.join([w for w in line.split() if len(w)>1])    
    
    return line


Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [27]:
data_df['tweet_token'] = data_df['tidy_tweet'].apply(lambda x: x.split())
#data_df.head()


,id,label,tweet,tidy_tweet,tweet_token
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, cannot, use, cause..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"


Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
#!python -m nltk.downloader stopwords

In [30]:
stop_words = nltk.corpus.stopwords.words('english')
data_df['tweet_token_filtered'] = data_df.tweet_token.progress_apply(lambda x: [w for w in x if w not in stop_words])
#data_df.tail()

100%|██████████| 49159/49159 [00:01<00:00, 32655.28it/s]


,id,label,tweet,tidy_tweet,tweet_token,tweet_token_filtered
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory left right polarisation trump ...,"[thought, factory, left, right, polarisation, ...","[thought, factory, left, right, polarisation, ..."
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,..."
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohio omg used word...,"[hillary, campaigned, today, in, ohio, omg, us...","[hillary, campaigned, today, ohio, omg, used, ..."
49157,49158,NaN,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download shoegaze newmusi...,"[my, song, so, glad, free, download, shoegaze,...","[song, glad, free, download, shoegaze, newmusi..."


Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [33]:
stemmer = nltk.stem.PorterStemmer()
 
data_df['tweet_stemmed'] = data_df.tweet_token_filtered.progress_apply(lambda x: [stemmer.stem(i) for i in x])
#data_df.head()


100%|██████████| 49159/49159 [00:10<00:00, 4553.66it/s]


,id,label,tweet,tidy_tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, cannot, use, cause...","[thanks, lyft, credit, cannot, use, cause, off...","[thank, lyft, credit, cannot, use, caus, offer..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"


Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [37]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [38]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
data_df['tweet_lemmatized'] = data_df.tweet_token_filtered.progress_apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

100%|██████████| 49159/49159 [00:03<00:00, 13278.38it/s]


Сохраним результат предобработки в pickle-файл.


In [39]:
data_df.to_pickle('/content/drive/MyDrive/Collab/Введение в обработку естественного языка/tweet_data.pkl')